In [ ]:
import os
from pathlib import Path
import pandas as pd


DATA_DIR = Path("./data")
DATA_SOURCE = Path(os.environ.get("HARVEST_DATA", "./data/harvest-sample.csv"))

In [ ]:
source = pd.read_csv(DATA_SOURCE)
source.head()